In [2]:
import os
import ee
import time
import re
from six.moves.urllib.request import urlretrieve
import numpy as np
import datetime
from tabulate import tabulate
from sortedcontainers import SortedDict
import csv
import itertools
import math
from IPython.display import Image

In [3]:
ee.Initialize()

In [4]:
image = ee.Image('srtm90_v4')
Image(url=image.getThumbUrl({'min':0, 'max':3000}))

In [6]:
# load the expanded  images
image = ee.Image("users/rosamaguilar/hcsfeats/sintext054112895010_01_ML_Sukumba_2014Y10M18D_WorldView-2_ORStandard2A_fused")
image2 = ee.Image("users/rosamaguilar/hcsfeats/sintext054112895020_01_ML_Sukumba_2014Y11M01D_WorldView-2_ORStandard2A_fused")
image3 = ee.Image("users/rosamaguilar/hcsfeats/sintext054112895030_01_ML_Sukumba_2014Y11M14D_WorldView-2_ORStandard2A_fused")
image4 = ee.Image("users/rosamaguilar/hcsfeats/sintext054112895040_01_ML_Sukumba_2014Y06M26D_WorldView-2_ORStandard2A_fused")
image5 = ee.Image("users/rosamaguilar/hcsfeats/sintext054112895070_01_ML_Sukumba_2014Y07M29D_WorldView-2_ORStandard2A_fused")
image6 = ee.Image("users/rosamaguilar/hcsfeats/sintext054330672010_01_ML_Sukumba_2014Y05M30D_WorldView-2_ORStandard2A_fused")
image7 = ee.Image("users/rosamaguilar/hcsfeats/sintext054330675010_01_ML_Sukumba_2014Y05M22D_WorldView-2_ORStandard2A_fused")

In [13]:
Image(url=image.getThumbUrl({'min':0, 'max':3000}))

In [7]:
Image(url=image2.getThumbUrl({min: 0, max: 2048, 'bands': 'b7,b4,b2', 'gamma': '1.1, 1.1, 1.1'}))

In [1]:
Image(url=image3.getThumbUrl({min: 0, max: 1024, 'bands': 'b5,b3,b2', 'gamma': '1, 1.1, 1.1'}))

NameError: name 'Image' is not defined

In [74]:
IC_toScaled = image.addBands(image2).addBands(image3).addBands(image4).addBands(image5).addBands(image6).addBands(image7)
Image(url=IC_toScaled.getThumbUrl({min: 0, max: 1024, 'bands': 'b5,b3,b2', 'gamma': '1.1, 1.1, 1.1'}))
print(IC_toScaled.bandNames().getInfo())

['fmask', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'NDI_b3_b2', 'NDI_b4_b2', 'NDI_b5_b2', 'NDI_b6_b2', 'NDI_b7_b2', 'NDI_b4_b3', 'NDI_b5_b3', 'NDI_b6_b3', 'NDI_b7_b3', 'NDI_b5_b4', 'NDI_b6_b4', 'NDI_b7_b4', 'NDI_b6_b5', 'NDI_b7_b5', 'NDI_b7_b6', 'DVI_b3_b2', 'DVI_b4_b2', 'DVI_b5_b2', 'DVI_b6_b2', 'DVI_b7_b2', 'DVI_b4_b3', 'DVI_b5_b3', 'DVI_b6_b3', 'DVI_b7_b3', 'DVI_b5_b4', 'DVI_b6_b4', 'DVI_b7_b4', 'DVI_b6_b5', 'DVI_b7_b5', 'DVI_b7_b6', 'RVI_b3_b2', 'RVI_b4_b2', 'RVI_b5_b2', 'RVI_b6_b2', 'RVI_b7_b2', 'RVI_b4_b3', 'RVI_b5_b3', 'RVI_b6_b3', 'RVI_b7_b3', 'RVI_b5_b4', 'RVI_b6_b4', 'RVI_b7_b4', 'RVI_b6_b5', 'RVI_b7_b5', 'RVI_b7_b6', 'EVI', 'TCARI', 'SAVI', 'MSAVI2', 'VARI', 'GLI', 'b2_1', 'b3_1', 'b4_1', 'b5_1', 'b6_1', 'b7_1', 'NDI_b3_b2_1', 'NDI_b4_b2_1', 'NDI_b5_b2_1', 'NDI_b6_b2_1', 'NDI_b7_b2_1', 'NDI_b4_b3_1', 'NDI_b5_b3_1', 'NDI_b6_b3_1', 'NDI_b7_b3_1', 'NDI_b5_b4_1', 'NDI_b6_b4_1', 'NDI_b7_b4_1', 'NDI_b6_b5_1', 'NDI_b7_b5_1', 'NDI_b7_b6_1', 'DVI_b3_b2_1', 'DVI_b4_b2_1', 'DVI_b5_b2_1', 'D

In [66]:
perc = 0
# function to scale values between 0 and 1
## function to transform the image to 255 bits (uint8):
#def transformTouint8Reducers(image, perc):

# This scale consider all values at the same time ?
# how to apply a reducer per band
NameBands = IC_toScaled.bandNames()

In [65]:
extrema = IC_toScaled.reduce(ee.Reducer.minMax())
print(extrema.getInfo())
m0 = ee.Number(extrema.get('min'))
m0.getInfo()

{'bands': [{'crs': 'EPSG:32614', 'id': 'min', 'data_type': {'min': 0, 'precision': 'int', 'max': 255, 'type': 'PixelType'}, 'crs_transform': [30.0, 0.0, 571785.0, 0.0, -30.0, 3140715.0]}, {'crs': 'EPSG:32614', 'id': 'max', 'data_type': {'min': 0, 'precision': 'int', 'max': 255, 'type': 'PixelType'}, 'crs_transform': [30.0, 0.0, 571785.0, 0.0, -30.0, 3140715.0]}], 'type': 'Image', 'properties': {'system:footprint': {'coordinates': [[[-97.7964, 27.300800000000002], [-96.99639999999998, 27.300800000000002], [-96.99639999999998, 28.100800000000003], [-97.7964, 28.100800000000003], [-97.7964, 27.300800000000002]]], 'geodesic': True, 'type': 'Polygon'}}}


In [68]:
geom = image2.geometry()
minValues = IC_toScaled.reduceRegion(
          reducer=ee.Reducer.max(),
          bestEffort=True,
          maxPixels=100000000).toImage(NameBands)      # to pairwise with the names of bands and use later 

maxValues = IC_toScaled.reduceRegion(
          reducer=ee.Reducer.percentile([100]),
          bestEffort=True,
          maxPixels=100000000).toImage(NameBands)

IC = (IC_toScaled.subtract(minValues).divide(maxValues.subtract(minValues))).toUint8()   #scale to 0-1
IC = IC.clip(geom)

SyntaxError: unexpected EOF while parsing (<ipython-input-69-d8e8d6cf05f5>, line 3)

In [72]:
Image(url=IC_toScaled.getThumbUrl({min: 0, max: 1, 'bands': 'b5,b3,b2', 'gamma': '1, 1.1, 1.1'}))

In [ ]:
classifiers = ({'CART':True, 'RF':True, 'SVML': True, 'SVMP': True, 'GMO': True})
weigth_classifiers = ({'CART':[0.94125],'GMO':[0.61625],'RF':[0.958125]